In [ ]:
import sys
path = '/gpfs/commons/groups/gursoy_lab/mstoll/'
sys.path.append(path)

import pandas as pd
import numpy as np 
import torch
import tensorboard
import os
import pickle
from torch.utils.tensorboard import SummaryWriter


from codes.models.data_form.DataForm import DataTransfo_1SNP, PatientList, get_paths_pheno_dicts
import matplotlib.pyplot as plt

In [ ]:
### data constants:
CHR = 1
SNP = 'rs673604'
pheno_method = 'Abby' # Paul, Abby
rollup_depth = 4
Classes_nb = 2 #nb of classes related to an SNP (here 0 or 1)
vocab_size = None # to be defined with data
padding_token = 0
prop_train_test = 0.8
load_data = True
save_data = False
remove_none = True
decorelate = False
equalize_label = False
threshold_corr = 0.9
threshold_rare = 50
remove_rare = 'all' # None, 'all', 'one_class'
compute_features = True
padding = True
list_env_features = ['age', 'sex']
### data format
batch_size = 20
data_share = 1

In [ ]:
dataT = DataTransfo_1SNP(SNP=SNP,
                         CHR=CHR,
                         method=pheno_method,
                         padding=padding,  
                         pad_token=padding_token, 
                         load_data=load_data, 
                         save_data=save_data, 
                         compute_features=compute_features,
                         prop_train_test=prop_train_test,
                         remove_none=True,
                         equalize_label=equalize_label,
                         rollup_depth=rollup_depth,
                         decorelate=decorelate,
                         threshold_corr=threshold_corr,
                         threshold_rare=threshold_rare,
                         remove_rare=remove_rare, 
                         list_env_features=list_env_features,
                         data_share=data_share)
#patient_list = dataT.get_patientlist()


In [ ]:
patient_list = dataT.get_patientlist()

In [ ]:
patient_list[0].SNP_label

In [ ]:
u = dataT.get_data_tabtransfo(patient_list=patient_list, actualise_phenos=False)

In [ ]:
u.keys()

In [ ]:
path = '/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/Training/SNPS/1/rs673604/Abby/dic_data_rs673604_decorelate=False.pkl'

In [ ]:
with open(path, 'rb') as file:
    o = pickle.load(file)

In [ ]:
o.keys()

In [ ]:
data, labels, indices_env, name_envs = dataT.get_tree_data(with_env=False)

In [ ]:
embeddings_file = '/gpfs/commons/groups/gursoy_lab/mstoll/codes/Data_Files/Embeddings/Abby/embedding_abby_no_1_diseases.pth'
embeddings = torch.load(embeddings_file)

In [ ]:
log_tensorboard_path= f'/gpfs/commons/groups/gursoy_lab/mstoll/codes/logs/plots/patients/tensorboard/patients_{CHR}_{SNP}'

In [ ]:
os.makedirs(os.path.dirname(log_tensorboard_path), exist_ok=True)

In [ ]:
embeddings_not_padded = embeddings[1:]

In [ ]:
patient_data_list = [embeddings_not_padded[list_diseases].mean(axis=0) for list_diseases in data]
patient_data_list = np.array(patient_data_list)

In [ ]:
patient_data_list = np.array(patient_data_list)


In [ ]:
int = 100000
patient_data_list_final = patient_data_list[:1000]
labels_final = labels[:1000]

In [ ]:
metadata = np.zeros((patient_data_list_final.shape[0],2), dtype=object)
metadata[:,1] = labels_final.astype(np.int64).astype(str)
metadata[:,0] = np.arange(patient_data_list_final.shape[0]).astype(str)
metadata = [list(data) for data in metadata]

In [ ]:
writer = SummaryWriter(log_tensorboard_path)

In [ ]:
writer.add_embedding(patient_data_list_final,metadata=metadata,  metadata_header=["Name","Label"])

In [ ]:
#### get patient_list

In [ ]:
path_pheno_id_dict, path_pheno_name_dict, path_pheno_cat_dict = get_paths_pheno_dicts(pheno_method, 4)
with open(path_pheno_id_dict, 'rb') as file:
    pheno_id_dict = pickle.load(file)
with open(path_pheno_name_dict, 'rb') as file:
    pheno_name_dict = pickle.load(file)
with open(path_pheno_cat_dict, 'rb') as file:
    pheno_cat_dict = pickle.load(file)
dic_pheno_reverse = {}
for key in pheno_id_dict:
    dic_pheno_reverse[pheno_id_dict[key]] = key

In [ ]:
def get_list_names(id):
    patient_list_data = np.argwhere(data[id]==1)
    list_phenos = []
    for pheno_id in patient_list_data:
        list_phenos.append(pheno_name_dict[dic_pheno_reverse[pheno_id[0]]])
    return list_phenos

In [ ]:
get_list_names(528)

In [ ]:
get_list_names(117) 

In [ ]:
get_list_names(402) 

In [ ]:
get_list_names(95332)
